In [1]:
%cd ..

/misc/vlgscratch4/LakeGroup/wentao/multimodal-baby


In [2]:
import itertools
import functools
from collections import namedtuple, Counter
import re
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.multimodal_data_module import PAD_TOKEN_ID, UNK_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID
from ngram import NGramModel
from multimodal.utils import map_structure
from analysis_tools.processing import examples_from_batches, get_pos_tags, get_word_pos_cnt, get_word_pos_stat_from_word_pos_cnt, get_pos_stats_for_words, get_model_losses_on_batches
from analysis_tools.sumdata import *
from analysis_tools.build_data import build_data
from analysis_tools.pos_tags import *
from analysis_tools.utils import *
from analysis_tools.word_categories import *
from analysis_tools.checkpoints import *


figsize = (13, 12)
sns.set_theme(
    style='white',
    font='monospace',
    rc={
        'figure.figsize': figsize,
        'axes.grid': False,
        'xtick.bottom': False,
        'xtick.top': False,
        'ytick.left': False,
        'ytick.right': False,
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
    }
)

np.set_printoptions(suppress=True, precision=2, linewidth=120)
pd.options.display.width = 120

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos_mapping = pos_mappings['syntactic category']

used_poses = ["noun", "verb", "adjective", "adverb", "function word", "cardinal number", "."][:2]

is_robustness_check = False

In [3]:
def pos_tagged_seq_repr(tokens, pos_tags):
    return ' '.join(f'{token}/{pos}' for token, pos in zip(tokens, pos_tags))


class Template(namedtuple('TemplateTuple', ['seq', 'pos', 'idx'])):
    """A template where the token at idx of seq is to be filled.
    """

    def __hash__(self):
        return hash((tuple(self.seq[:self.idx]), tuple(self.seq[self.idx+1:])))

    def __str__(self):
        return ' '.join(
            '_' * len(token) if i == self.idx else token
            for i, (token_id, token) in enumerate(
                (token_id, idx2word[token_id]) for token_id in self.seq)
            if token_id not in [SOS_TOKEN_ID, EOS_TOKEN_ID])


def template_from_str(s, use_unk=False):
    """Construct Template from str s. This is for manual construction of templates.
    Inputs:
        s: str. A sentence that is tokenized (tokens are separated by space) and containing exactly one slot, represented by '_'.
            Example: "here 's a _ and a kitty ."
    Return:
        a Template.
    """
    tokens = s.split()
    token_ids = []
    idx = None
    for i, token in enumerate(tokens):
        if re.fullmatch(r'_+', token):
            if idx is not None:
                raise Exception(f'more than one slot in template: {s}')
            idx = i
            token_id = UNK_TOKEN_ID
        else:
            try:
                token_id = word2idx[token]
            except KeyError:
                if use_unk:
                    token_id = UNK_TOKEN_ID
                else:
                    raise
        token_ids.append(token_id)
    if idx is None:
        raise Exception(f'must have one slot in template: {s}')

    token_ids = [SOS_TOKEN_ID] + token_ids + [EOS_TOKEN_ID]
    idx += 1
    pos = ["."] * len(token_ids)
    return Template(token_ids, pos, idx)


def templates_from_example(token_ids, pos_tags):
    for i, (token_id, pos) in enumerate(zip(token_ids, pos_tags)):
        if pos_mapping[pos] in used_poses:
            yield Template(token_ids, pos_tags, i)


def templates_from_examples(examples, pos_tags, print_pos_tagged_seq=False):
    tot, cnt_filtered = 0, 0

    for example, y_pos_tags in zip(examples, pos_tags):
        x, y, y_len, raw_y = example
        tot += 1
        if print_pos_tagged_seq:
            print(pos_tagged_seq_repr(raw_y[0].split(), y_pos_tags[1:]))
        cnt_present_tokens = sum((pos_mapping[pos] not in ["."] for pos in y_pos_tags))
        if cnt_present_tokens <= 2:
            continue
        y_len = y_len.item()
        y = y[:y_len].tolist()
        if UNK_TOKEN_ID in y:
            continue

        cnt_filtered += 1
        yield example, templates_from_example(y, y_pos_tags)

    print(f'filtered {cnt_filtered} / {tot} = {cnt_filtered / tot :.2%} examples')


def joined_templates_from_examples(*args, **kwargs):
    for example, example_templates in templates_from_examples(*args, **kwargs):
        for template in example_templates:
            yield example, template


def construct_batch_by_filling_template(template, word_ids, **kwargs):
    batch_size = len(word_ids)
    y = torch.tensor(template.seq)
    y_batch = y.unsqueeze(0).repeat(batch_size, *([1] * y.dim()))
    y_batch[:, template.idx] = torch.tensor(word_ids)
    y_len = torch.tensor(len(template.seq))
    y_len_batch = y_len.unsqueeze(0).expand(batch_size, *([-1] * y_len.dim()))
    return {
        'y': y_batch,
        'y_len': y_len_batch,
        **{
            key: (value.unsqueeze(0).expand(batch_size, *([-1] * value.dim())) if value is not None else None)
            for key, value in kwargs.items()
        }
    }


def run_model_on_template(model, example, template, word_ids, batch_size=256):
    """Run model and get the whole sentence losses on template filled with every word_id in word_ids
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        batch_size: the batch size
    Returns:
        losses: an np.ndarray of length len(word_ids)
    """

    x, y, y_len, raw_y = example
    x = x.to(device=device)
    x = x.unsqueeze(0)
    if isinstance(model, MultiModalLitModel) and (
            model.language_model.text_encoder.captioning or
            model.language_model.text_encoder.has_attention):
        image_features, image_feature_map = model.model.encode_image(x)
        image_features = image_features.squeeze(0)
        image_feature_map = image_feature_map.squeeze(0)
    else:
        image_features, image_feature_map = None, None

    batches = (
        construct_batch_by_filling_template(
            template,
            word_ids[i : i + batch_size],
            image_features=image_features,
            image_feature_map=image_feature_map,
        )
        for i in range(0, len(word_ids), batch_size))
    return get_model_losses_on_batches(model, batches)


def find_index(lst, e):
    for i, x in enumerate(lst):
        if x == e:
            return i
    return None


def get_prob_dist(model, example, template, word_ids, word_cats, pos_mapping=identity, batch_size=256, top_k=None):
    """Run model on template and get the probability distribution of categories;
    check whether model gives higher probability to the correct category.
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        word_cats: np.ndarray of same length as word_ids; the categories of each word in word_ids
        pos_mapping: callable, used to map pos to cat
        batch_size: the batch size used to run the model
        top_k: print top k predictions along with the ground-truth label; None if unwanted
    Returns:
        pd.Series with categories indexing the probability distribution
    """

    losses = run_model_on_template(model, example, template, word_ids, batch_size=batch_size)
    probs = F.softmax(-losses, 0)

    gt_word_id = template.seq[template.idx]
    gt_word_pos = template.pos[template.idx]
    gt_word_cat = pos_mapping(gt_word_pos)
    gt_word_idx = find_index(word_ids, gt_word_id)
    if gt_word_idx is None:  # the ground-truth word is not in the vocab of prediction
        print(f'ground-truth word {idx2word[gt_word_id]} not in the vocab of prediction')
        # append the ground-truth word to the vocab
        gt_word_idx = len(word_ids)
        word_ids = np.append(word_ids, gt_word_id)
        word_cats = np.append(word_cats, gt_word_cat)
        losses = torch.cat([losses, torch.tensor([np.inf], dtype=losses.dtype, device=losses.device)])
        probs = torch.cat([probs, torch.tensor([0.], dtype=probs.dtype, device=probs.device)])
    else:
        # set the category of the ground-truth word to the category according to the template
        if word_cats[gt_word_idx] != gt_word_cat:
            word_cats = np.copy(word_cats)
            word_cats[gt_word_idx] = gt_word_cat

    if top_k is not None:
        words = idx2word[word_ids]
        print_losses = False
        if print_losses:
            print_top_values(-losses, words, labels=torch.tensor(gt_word_idx), value_formatter=lambda loss: f'{-loss:6.3f}')
        print_top_values(probs, words, labels=torch.tensor(gt_word_idx), value_formatter=prob_formatter)

    probs_ser = pd.Series(probs.cpu())
    return probs_ser.groupby(word_cats).sum()

In [4]:
# build data and vocab according to the model
data, args = build_data(return_args=True)
dataset_name = args.dataset
vocab = data.datasets['train'].vocab
vocab_size = len(vocab)
print(f'{vocab_size = }')
word2idx = vocab
idx2word = [None] * vocab_size
for word, idx in word2idx.items():
    idx2word[idx] = word
idx2word = np.array(idx2word)

my_batch_size = 256
dataloader_fns = {
    'train': lambda: data.train_dataloader(batch_size=my_batch_size, shuffle=False, drop_last=False),
    'val': lambda: data.val_dataloader(batch_size=my_batch_size)[0],
    'test': lambda: data.test_dataloader(batch_size=my_batch_size)[0],
}

# load model from checkpoint

# select from list of checkpoints
dataset_name = "saycam"
names = [
    "LSTM 0", "LSTM 1", "LSTM 2",
    "CBOW 0", "CBOW 1", "CBOW 2",
    "Captioning LSTM 0", "Captioning LSTM 1", "Captioning LSTM 2",
]
dataset_checkpoint_paths = all_checkpoint_paths[dataset_name]
checkpoint_paths = [dataset_checkpoint_paths[name] for name in names]

models = []

for checkpoint_path in checkpoint_paths:
    print(f"load model from {checkpoint_path}")
    lit_model = MultiModalLitModel.load_from_checkpoint(checkpoint_path, map_location=device)
    lit_model.to(device)
    lit_model.eval()
    model = lit_model

    models.append(model)

# get the pos tags of all words in vocab
train_dataloader_fn = dataloader_fns['train']
train_pos_tags = get_pos_tags(train_dataloader_fn(), dataset_name, 'train')
word_pos_cnt = get_word_pos_cnt(train_dataloader_fn(), train_pos_tags)
word_pos_stat = get_word_pos_stat_from_word_pos_cnt(word_pos_cnt)
pos_stats = get_pos_stats_for_words(idx2word, word_pos_stat, pos_mapping=pos_mapping.get)
idx2pos2 = np.array([pos_stat[0][0] if pos_stat else '.' for pos_stat in pos_stats])

for pos in sorted(set(pos_mapping.values())):
    pos_word_ids = np.nonzero(idx2pos2 == pos)[0]
    print(f'#{pos:15s}: {len(pos_word_ids)}')

idx2used = np.isin(idx2pos2, used_poses)

for token_id in [PAD_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID, UNK_TOKEN_ID]:
    idx2used[token_id] = False

for idx, (word, pos_stat) in enumerate(zip(idx2word, pos_stats)):
    used_pos_stat = [(pos, cnt) for pos, cnt in pos_stat if pos in used_poses]
    if not used_pos_stat:
        continue
    word_cnt = sum(cnt for pos, cnt in used_pos_stat)
    stat_str = ' '.join(f'{pos:15} {cnt:5} {cnt / word_cnt :6.1%}' for pos, cnt in used_pos_stat)
    print(f'{word:10s} {stat_str}')

    # remove too ambiguous words
    if used_pos_stat[0][1] / word_cnt < 0.9:
        idx2used[idx] = False

# untypical words
my_untypical_words = ' '.join(
    possessives + negations + be_verbs + do_verbs + modal_verbs +
    pronoun_contractions + other_contractions + quantifiers + pos_ambiguous_words +
    special_tokens
).split()
my_untypical_word_ids = [word2idx[word] for word in my_untypical_words if word in word2idx]

# whether to excluding untypical words from the filling word vocabulary
not_filling_untypical_words = is_robustness_check
if not_filling_untypical_words:
    for word_id in my_untypical_word_ids:
        idx2used[word_id] = False

# get filling word vocabulary
word_ids = np.nonzero(idx2used)[0]
word_cats = idx2pos2[word_ids]
used_vocab_size = len(word_ids)
word_cat_counter = Counter(word_cats)
print(f'{used_vocab_size = } {word_cat_counter}')

Using base transforms
Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created. Skipping this step.
Shuffled training metadata file has already been created. Skipping this step.
Evaluation frames have already been filtered. Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Filtered evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Vocabulary file already exists. Skipp

/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'vision_encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['vision_encoder'])`.
  rank_zero_warn(
/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'text_encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['text_encoder'])`.
  rank_zero_warn(


load model from checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_1/epoch=38.ckpt
load model from checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_2/epoch=28.ckpt
load model from checkpoints/lm_dataset_saycam_text_encoder_cbow_embedding_dim_512_tie_False_bias_False_crange_1_dropout_i_0.0_dropout_o_0.1_batch_size_8_lr_0.003_lr_scheduler_True_patience_2_weight_decay_0.04_seed_0/epoch=31.ckpt
load model from checkpoints/lm_dataset_saycam_text_encoder_cbow_embedding_dim_512_tie_False_bias_False_crange_1_dropout_i_0.0_dropout_o_0.1_batch_size_8_lr_0.003_lr_scheduler_True_patience_2_weight_decay_0.04_seed_1/epoch=65.ckpt
load model from checkpoints/lm_dataset_saycam_text_encoder_cbow_embedding_dim_512_tie_False_bias_False_crange_1_dropout_i_0.0_

In [5]:
# observation: many ambiguous cases seem to be confusing nouns vs V-ings/passive verbs.


printing_labels = False


splits = ['val']

filter_out_gt_word_ids = [SOS_TOKEN_ID, EOS_TOKEN_ID]
if is_robustness_check:
    filter_out_gt_word_ids += my_untypical_word_ids

seen_templates = set()
for split in ['train']:
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_batches(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    seen_templates.update(template for example, template in joined_templates_from_examples(examples, pos_tags))

for split in splits:
    print(f'{split} split:')
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_batches(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    golds, preds = [], {name: [] for name in names}
    for example, template in joined_templates_from_examples(examples, pos_tags):
        gt_word_id = template.seq[template.idx]
        if gt_word_id in filter_out_gt_word_ids:
            continue
        if not idx2used[gt_word_id]:
            continue
        print(template)
        if template in seen_templates:
            print('seen template; skipped')
            continue

        gold = pos_mapping[template.pos[template.idx]]
        golds.append(gold)
        for model_name, model in zip(names, models):
            prob_dist = get_prob_dist(model, example, template, word_ids, word_cats, pos_mapping=pos_mapping.get, top_k=5)
            if printing_labels:
                print(prob_dist)
            argmax_idx = prob_dist.argmax()
            pred = prob_dist.index[argmax_idx]
            preds[model_name].append(pred)
            correct = pred == gold
            if printing_labels:
                print(f'{gold = }, {pred = }, {correct = }')

    print(f'label distribution: ')
    golds = np.array(golds)
    for pos in used_poses:
        cnt = (golds == pos).sum()
        print(f'{pos}: {frac_format(cnt, len(golds))}')
    for model_name, model in zip(names, models):
        preds[model_name] = np.array(preds[model_name])
        corrects = preds[model_name] == golds
        print(f'{model_name} accuracy: {frac_format(corrects.sum(), len(corrects))}')

load cached pos tags: dataset_cache/saycam/train.pos.cache
filtered 20379 / 33737 = 60.41% examples
val split:
load cached pos tags: dataset_cache/saycam/val.pos.cache
we should ____ on some lights , huh ?
  5.2% turn     |  91.2% put        5.2% turn       0.4% leave      0.4% keep       0.4% get     
 64.1% turn     |  64.1% turn      19.2% put        2.6% go         1.9% read       1.7% blow    
 16.4% turn     |  49.9% put       23.2% go        16.4% turn       1.2% sit        1.1% leave   
  1.1% turn     |  48.2% put       31.4% lid        8.9% go         2.3% sit        1.9% come    
  1.2% turn     |  48.8% put       27.6% lid       11.7% go         2.0% come       1.8% sit     
  1.7% turn     |  51.8% put       18.1% lid       12.7% go         3.8% sit        2.6% hold    
 10.7% turn     |  72.5% put       10.7% turn       4.8% blow       2.8% hang       1.8% go      
  8.9% turn     |  81.0% put        8.9% turn       5.5% go         1.1% look       0.5% leave   
  0.7% tur

 12.9% do       |  12.9% do        12.6% make      10.8% get       10.1% blow       7.2% have    
we can __ walk over there if you want .
 43.4% go       |  43.4% go         9.4% come       7.7% have       5.6% do         2.9% be      
 85.4% go       |  85.4% go         2.2% come       1.9% do         1.0% try        1.0% pretend 
 47.7% go       |  47.7% go         5.6% do         3.9% have       3.6% be         3.3% play    
 43.5% go       |  43.5% go         9.6% play       5.5% come       5.1% dig        3.3% make    
 38.8% go       |  38.8% go         8.8% play       7.5% come       5.0% dig        4.3% have    
 44.6% go       |  44.6% go        11.7% play       6.7% come       6.3% dig        2.4% have    
 30.2% go       |  30.2% go        15.4% try        9.9% have       9.6% do         7.3% dig     
 40.2% go       |  40.2% go         9.2% dig        6.4% be         4.3% keep       3.6% have    
 88.8% go       |  88.8% go         1.0% try        0.9% dig        0.9% do   

 31.9% go       |  56.9% see       31.9% go         7.0% try        0.7% cross      0.5% play    
 19.6% go       |  67.2% see       19.6% go         5.8% try        1.2% do         0.9% play    
 37.7% go       |  37.7% go         5.8% see        3.6% sam        3.0% one        2.7% natalie 
 38.9% go       |  38.9% go         5.0% see        3.1% sam        3.0% one        2.5% natalie 
 39.7% go       |  39.7% go         5.1% see        3.2% sam        2.8% one        2.3% jam     
 69.1% go       |  69.1% go        24.9% see        1.7% try        1.2% cross      0.4% wait    
 41.0% go       |  49.2% see       41.0% go         4.6% try        0.9% cross      0.3% get     
 34.6% go       |  58.3% see       34.6% go         2.7% cross      1.6% try        0.5% play    
that __ an o !
 76.8% 's       |  76.8% 's         7.9% ’s         5.8% s          4.7% is         3.6% was     
 87.6% 's       |  87.6% 's         4.8% ’s         3.3% s          2.3% is         0.9% has     
 84.7

  9.2% banana   |  14.3% bottle     9.2% banana     8.9% puzzle     5.2% bag        3.6% nectarine
  4.5% banana   |   7.6% bottle     4.6% kitty      4.5% banana     4.4% book       3.7% pen     
there we __ , now can you help me , did you want to help me pick out some clothes look we have new
 99.8% go       |  99.8% go         0.1% are        0.0% have       0.0% do         0.0% 're     
 99.1% go       |  99.1% go         0.3% are        0.1% do         0.1% have       0.0% want    
 99.7% go       |  99.7% go         0.2% are        0.0% have       0.0% 're        0.0% need    
 54.3% go       |  54.3% go        12.4% do        11.9% are        1.9% put        1.8% have    
 53.5% go       |  53.5% go        12.6% are       12.3% do         2.0% put        1.5% know    
 53.9% go       |  53.9% go        12.4% do        12.3% are        1.9% put        1.8% have    
 99.7% go       |  99.7% go         0.0% do         0.0% see        0.0% are        0.0% use     
 99.9% go       | 

  0.3% keep     |  73.1% put       13.9% putting    0.8% check      0.8% like       0.7% take    
  0.1% keep     |  70.4% be        14.3% 's         4.4% is         2.6% make       1.1% says    
  0.1% keep     |  78.6% be        10.1% 's         2.2% is         1.6% make       1.2% says    
  0.1% keep     |  70.9% be         9.6% 's         4.6% is         4.0% make       1.6% says    
  0.4% keep     |  89.0% put        2.6% putting    1.4% push       1.4% got        1.1% pour    
  0.0% keep     |  87.2% putting    9.3% put        0.6% get        0.4% like       0.3% getting 
  0.2% keep     |  43.0% putting   22.1% put        4.6% is         3.0% taking     1.7% getting 
oh whoops you want to be very careful keep it in the ____ .
  5.3% bowl     |  14.9% bin       13.8% mirror     9.6% pantry     7.0% bathtub    5.3% bowl    
  4.1% bowl     |  18.2% mirror    13.5% pantry     6.3% car        4.1% bowl       3.9% sink    
  3.7% bowl     |  11.4% box        9.6% mirror     4.8% c

...

 85.0% is       |  85.0% is        10.9% 's         1.2% has        1.2% was        0.5% have    
 59.3% is       |  59.3% is         8.1% one        4.9% have       3.9% salt       3.4% had     
 55.7% is       |  55.7% is        11.5% one        5.5% have       3.4% had        3.3% salt    
 61.3% is       |  61.3% is        10.6% one        5.4% have       3.3% had        2.2% eating  
 95.7% is       |  95.7% is         1.8% 's         0.8% s          0.7% has        0.4% have    
 96.4% is       |  96.4% is         2.7% 's         0.3% are        0.1% goes       0.1% have    
 95.6% is       |  95.6% is         1.1% 's         1.0% are        0.6% tastes     0.5% has     
hopefully this is enough in here otherwise i ____ to warm up some more water .
  5.9% have     |  88.4% want       5.9% have       1.8% need       0.9% wants      0.6% use     
 64.9% have     |  64.9% have      18.4% need       8.5% want       1.8% forgot     1.7% 've     
 67.5% have     |  67.5% have      13.2

  8.9% turn     |  55.1% put       14.2% hold       8.9% turn       5.3% go         4.2% draw    
  8.1% turn     |  55.7% put       15.3% hold       8.1% turn       4.7% go         4.0% draw    
 71.8% turn     |  71.8% turn       8.7% put        5.3% blow       3.6% stand      1.5% sit     
 25.6% turn     |  59.0% put       25.6% turn       2.2% leave      1.2% be         1.1% get     
 97.1% turn     |  97.1% turn       1.1% put        0.6% stand      0.2% choke      0.2% go      
wanna __ something ?
  9.9% do       |  25.9% read       9.9% do         9.8% get        8.7% draw       8.5% try     
 19.1% do       |  19.1% do        17.0% read      13.9% try        7.3% draw       6.4% spell   
 25.3% do       |  25.3% do        18.9% draw      10.9% read       5.7% get        3.4% press   
  7.6% do       |  28.1% draw      24.0% read      10.2% try        7.6% do         5.8% spell   
  7.4% do       |  30.3% read      27.8% draw      10.5% try        7.4% do         4.4% get     

  1.0% standing |  87.4% sitting    4.4% going      2.9% turning    1.3% putting    1.0% standing
 15.0% standing |  42.6% sitting   15.5% turning   15.0% standing   8.1% swinging   6.4% going   
 64.0% standing |  64.0% standing  16.6% sitting    3.3% going      2.7% swinging   1.7% choke   
 51.3% standing |  51.3% standing   9.7% sitting    5.2% swinging   3.9% turning    3.4% going   
what __ outside ?
 59.2% 's       |  59.2% 's        15.0% s         11.6% is         2.9% are        2.4% happened
 82.5% 's       |  82.5% 's         6.2% is         5.4% s          0.9% happened   0.9% are     
 72.6% 's       |  72.6% 's        13.0% s          6.8% is         0.8% went       0.7% go      
 10.1% 's       |  33.5% look      33.2% see       10.1% 's         8.4% is         6.7% are     
 11.0% 's       |  39.7% see       21.6% look      11.0% 's         8.4% is         6.7% are     
 11.8% 's       |  38.6% see       22.8% look      11.8% 's         8.9% is         6.8% are     
 6

  1.4% dummy    |  32.1% ball      10.2% banana     6.3% strawberry   5.9% bite       4.7% one     
  0.9% dummy    |  28.5% one       22.9% bite       4.7% ball       4.5% book       3.3% way     
  0.0% dummy    |  94.0% 's         2.2% is         0.9% find       0.5% bite       0.4% piece   
  0.0% dummy    |  94.0% 's         2.0% is         1.0% find       0.5% bite       0.3% put     
  0.0% dummy    |  93.9% 's         2.2% is         0.8% find       0.5% bite       0.3% put     
  7.8% dummy    |  18.8% one       16.2% bite      11.5% banana     7.8% dummy      4.8% semicolon
 17.5% dummy    |  51.5% one       17.5% dummy      3.8% bite       3.1% egg        2.0% semicolon
 24.6% dummy    |  36.5% piece     24.6% dummy     18.4% one        3.4% bite       1.7% ball    
you have another dummy it __ a wombat
  3.4% ’s       |  42.3% 's        27.1% has        6.4% got        4.9% is         3.4% looks   
  8.6% ’s       |  52.4% 's        23.8% has        8.6% ’s         6.0% is 

  0.4% belly    |  46.2% hands      5.6% shirt      5.5% shoes      5.4% pants      4.3% socks   
  0.2% belly    |  33.5% hands     13.8% plate      9.4% shirt      5.4% nappy      3.2% jacket  
  0.6% belly    |  65.0% hands      6.6% pants      5.3% pajamas    3.7% shirt      1.9% bike    
  0.2% belly    |  20.6% pajamas   13.4% jacket    13.0% crib      11.1% snot       8.9% leg     
  0.2% belly    |  18.4% crib      17.2% pajamas   13.0% leg        6.9% jacket     6.8% snot    
  0.3% belly    |  20.9% crib      15.4% pajamas   12.2% snot       8.8% leg        7.0% jacket  
  0.0% belly    |  29.2% hands     18.9% banana     8.6% spoon      4.6% shirt      4.0% bowl    
  0.2% belly    |  84.3% hands      2.3% banana     1.4% shirt      1.2% legs       1.0% teeth   
  0.1% belly    |  59.2% hands      3.8% socks      3.4% shirt      3.0% banana     2.6% pajamas 
we can wash your belly but not your _____ .
  1.3% shirt    |  14.5% face       8.2% nappy      8.0% feet       7.3% h

In [6]:
# TODO: examine whether BabyBERTa vocab is similar to SAYCam's; if true, we can try that dataset.


manual_templates = [
    "here 's _ .",
    "here 's a _ .",
    "here 's the _ .",
    "here 're _ .",
    "an _ .",
    "a _ .",
    "a kitty and a _ .",
    "a _ and a kitty .",
    "kitties and _ .",
    "_ and kitties .",
    "a doggy and a _ .",
    "a _ and a doggy .",
    "a cow and a _ .",
    "a _ and a cow .",
    "cows and _ .",
    "_ and cows .",
    "a train and a _ .",
    "a _ and a train .",
    "a car and a _ .",
    "a _ and a car .",
    "a banana and a _ .",
    "an apple and a _ .",
    "shoes and _ .",
    "_ and shoes .",
    "a jacket and a _ .",
    "a baby and a _ .",
    "a _ and a baby .",
    "have the _ .",
    "have some _ .",
    "take the _ .",
    "eat _ .",
    "eat the _ .",
    "drink _ .",
    "drink some _ .",
    "wear the _ .",
    "go to _ .",
    "go to the _ .",
    "get on the _ .",
    "get into the _ .",
    "the girl the boys love _ there .",
    "the girl that the boys see _ there .",
    "the girl that the boys saw _ there .",
    "the girl that the boys love _ there .",
    "the girl that the boys loved _ there .",
    "the boys the girl likes _ there .",
    "the boys the girl liked _ there .",
    "the boys the girl loved _ there .",
    "the boys that the girl saw _ there .",
    "the horse ran past the _ .",
    "the horse ran past the barn _ .",
    "the horse ran past the barn _",
    "the doggy ran past the _ .",
    "the birdie flew past the _ .",
]

x = torch.zeros_like(next(examples_from_batches(dataloader_fns['val']()))[0])

for template_str, template in zip(manual_templates, map(template_from_str, manual_templates)):
    print(template)
    y = torch.tensor(template.seq)
    y_len = torch.tensor(len(template.seq))
    example = x, y, y_len, [template_str]
    for model_name, model in zip(names, models):
        prob_dist = get_prob_dist(model, example, template, word_ids, word_cats, pos_mapping=pos_mapping.get, top_k=5)
        if printing_labels:
            print(prob_dist)

here 's _____ .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  38.1% one        5.2% bunny      3.4% grandpa    2.5% bread      2.3% mommy   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  17.1% one        4.2% bunny      4.1% natalie    3.9% sam        3.2% water   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  39.0% one        5.0% sam        4.1% flour      3.9% fiona      3.6% bunny   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  11.4% go         9.7% natalie    8.7% see        4.5% jam        4.5% coming  
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  11.7% go         9.4% natalie    7.8% see        4.8% jam        4.3% coming  
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  12.1% go         8.5% natalie    8.0% see        4.8% jam        3.9% coming  
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  11.0% lot        5.1% bottle     4.6% banana     4.0% kitty      3.7% bit     
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  19.1% kitty     15.8% bear       4.4% doggy      4.3% sock       2.3% duck    
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  26.7% kitty      6.1% bear       3.7% doggy      2.9% guitar     2.2% house   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  14.9% kitty     10.4% cow        7.7% doggy      4.8% bunny      4.1% bus     
a _____ and a kitty .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  31.7% kitty     20.8% doggy      6.1% horse      3.5% tiger      2.6% flower  
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  32.3% kitty     14.1% doggy     12.2% horse      7.5% flower     3.5% dinosaur
ground-truth word <unk> not in the vocab of prediction
  0.0

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  24.2% cow        6.2% sheep      5.0% bear       2.9% bicycle    2.7% bunny   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  59.2% cow        5.7% chicken    3.5% sheep      1.6% bunny      1.5% bus     
a _____ and a cow .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  19.4% horse     16.7% bunny      5.4% bucket     5.4% cow        3.6% sheep   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  22.0% bunny     21.4% horse      5.4% sheep      4.3% cow        3.8% bucket  
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  25.5% horse     11.8% flower     4.1% duck       3.6% bunny      2.7% ball    
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |   7.5% flower     5.5% duck       5.5% track      4.7% doggy      4.7% strawberry
ground-truth word <unk> not in the vocab of prediction
  0.0

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  52.0% flower     7.8% car        7.1% dinosaur   2.8% tiger      2.0% hat     
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  60.3% flower     6.5% car        2.3% train      2.3% ball       2.0% horse   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |   7.5% flower     5.5% duck       5.5% track      4.7% doggy      4.7% strawberry
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |   7.9% flower     5.9% doggy      5.6% track      5.0% strawberry   4.8% duck    
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |   7.8% flower     6.6% track      5.2% duck       4.9% strawberry   4.8% doggy   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  24.0% car       17.8% flower     7.6% lion       3.2% train      2.7% doggy   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  44

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  15.9% bear       6.2% girl       5.6% doggy      5.4% boy        5.3% baby    
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  10.2% bear       6.7% bottle     5.6% car        4.7% kitty      2.4% baby    
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |   7.6% bus        6.6% emily      6.1% baby       4.2% cow        3.2% lion    
a _____ and a baby .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  17.9% kitty     11.9% baby       6.1% train      5.4% boy        3.8% doggy   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  13.1% baby      11.2% duck       8.5% kitty      4.6% bear       4.4% doggy   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  23.5% dinosaur   6.8% boy        5.8% ball       5.5% baby       4.6% train   
ground-truth word <unk> not in the vocab of prediction
  0.0%

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  34.8% sam        4.4% water      4.3% one        2.5% shoes      2.0% coffee  
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |   9.7% bubbles    6.4% sam        3.7% water      3.5% poo        3.3% shoes   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  30.0% sam        7.0% water      4.1% milk       2.6% something   2.6% poo     
drink some _____ .
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  17.8% water     12.3% bubbles    9.5% milk       4.6% bread      3.7% grapes  
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  50.5% milk      11.6% water      4.7% bread      3.2% banana     2.7% juice   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  51.3% milk       7.0% bread      6.0% water      5.2% bubbles    3.0% music   
ground-truth word <unk> not in the vocab of prediction
  0.0% 

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  17.7% sitting   16.4% was       12.1% is        11.2% are        6.7% running 
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  70.9% is        19.6% does       4.2% 's         3.7% are        0.6% s       
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  64.7% is        19.9% does       6.8% 's         6.1% are        1.0% swim    
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  69.2% is        19.9% does       5.0% 's         4.5% are        0.4% s       
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  34.5% is        24.7% was        9.9% are        5.3% come       4.0% be      
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  50.0% is        16.5% was        7.3% are        2.6% coming     2.4% 's      
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |   9.2% is

ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  10.9% was       10.6% play       5.9% come       5.8% stay       5.5% juice   
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  44.4% is        13.0% something  11.2% holes      7.9% are        1.5% 's      
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  10.7% play       5.6% used       5.0% grapes     4.1% words      4.0% is      
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  58.3% are       18.3% 's         9.1% is         6.7% were       1.6% s       
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  52.0% are       26.0% 's        12.5% is         3.6% were       1.4% was     
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  68.8% are       10.1% is         9.6% 's         2.9% were       1.6% was     
ground-truth word <unk> not in the vocab of prediction
  0.0% <unk>    |  46.7% a